In [73]:
import pickle
import dask.dataframe as dd
import pandas as pd

In [74]:
eICU_features = set(pd.read_csv('../1_importsql/feature_df2.csv').concept_name)
feature_eicu = pd.read_csv('feature_eicu.csv')
feature_hicu = pd.read_csv('feature_hicu.csv')

## 매칭 안되는 부분
- drug : 세브란스에서 prescription은 쓸 수 없어서

## 매칭
- fuzzy algorithm 이용해서 비슷한 string에 대해서 매칭 진행하도록 한다.

In [82]:
feature_eicu.concept_table.unique()

array(['lab', 'adm'], dtype=object)

In [75]:
matching_field = feature_eicu[['concept_name']].merge(feature_hicu[['hicu_feature_english']], how='cross')

In [76]:
matching_field.head()

,concept_name,hicu_feature_english
0,-bands,address1
1,-bands,bp
2,-bands,Creatinine
3,-bands,body surface area
4,-bands,heightZ


In [77]:
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def find_matching(eicu_feature, hicu_feature):
    ratio = fuzz.partial_ratio(eicu_feature.lower(), hicu_feature.lower())
    return ratio

In [78]:
matching_field['match_score'] = matching_field.apply(lambda x : find_matching(x['concept_name'], x['hicu_feature_english']), axis=1)

In [79]:
matching_field.query('match_score > 80').to_csv('matching.csv', index=False)

일일이 매칭 확인 후 맵핑된 것을 matching.csv로 업로드 하였음.

In [83]:
matched = pd.read_csv('matching.csv')

In [87]:
matched[matched.matching == 1].reset_index(drop=True)

# 이외에 height, weight, vital sign까지 포함해서 common feature로 이용

,concept_name,hicu_feature_english,match_score,matching
0,BUN,BUN,100,1
1,CREATININE ...,Creatinine,100,1
2,GLUCOSE ...,Glucose,100,1
3,HCO3,HCO3-,100,1
4,Hct,Hct,100,1
5,POTASSIUM ...,potassium,100,1
6,RBC,RBC COUNT,100,1
7,Temperature,temperature,100,1
8,creatinine,Creatinine,100,1
9,glucose,Glucose,100,1


In [21]:
hicu_features

{'BUN',
 'Basophil(%)',
 'Cl',
 'Creatinine',
 'Eosinophil(%)',
 'Glucose(POCT)',
 'HCO3-',
 'Hct',
 'Hemoglobin',
 'K',
 'Monocyte(%)',
 'Na',
 'Neutrophil(%)',
 'PLT Count',
 'RBC COUNT',
 'aPTT',
 'idx',
 'pH',
 'pO2',
 '가진찰권번호여부',
 '검사일자',
 '구역번호',
 '국제수가_적용여부',
 '급여제한여부',
 '동의서진행상태',
 '동의서진행상태명',
 '맥박수',
 '무효처리_구분',
 '생년월',
 '성별',
 '성별명',
 '신장',
 '신장P',
 '신장Z',
 '연구내원번호',
 '연구등록번호',
 '연구신규_진찰권번호',
 '외국인여부',
 '입력일자',
 '입실나이',
 '전입_일주일전',
 '전입일자',
 '전출_일주일후',
 '전출일자',
 '조회시점나이',
 '주민번호',
 '주소1',
 '주소구분',
 '주소출력일자',
 '지역병원코드',
 '지역병원코드명',
 '체온',
 '체중',
 '체중P',
 '체중Z',
 '체표면적',
 '최근내원일',
 '최초_병원코드',
 '혈압',
 '호흡수',
 '환자정보비공개여부',
 '휴대폰_관계코드',
 '휴대폰전화번호1',
 '휴대폰전화번호2'}